In [21]:
import os
import warnings
import textwrap
os.environ['KPM_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings('ignore')

In [3]:
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import tiktoken

In [4]:
base_url = "http://localhost:11434"
model_name = 'nomic-embed-text'

embeddings = OllamaEmbeddings(model='phi3', base_url=base_url)

In [5]:
db_name = 'docs/health supplements/'
vector_store = FAISS.load_local(folder_path=db_name, embeddings=embeddings, allow_dangerous_deserialization=True)

In [6]:
question = 'how to gain muscle mass?'
docs = vector_store.search(query=question, k=5, search_type='similarity')

In [7]:
for a in docs:
    print('page: ', a.metadata.get('page'))
    print(a.page_content)
    print('\n')

page:  6
supplements is currently around 20%. Furthermore, bodybuilding and weight loss 
supplements account for almost half of these injuries (62). Among emergency department 
visits for adverse events related to dietary supplements in the US, around 25% were due to 
weight loss products (63). There are two classes of adverse effects that may occur. 
Supplements can have components according to the product description that cause certain 
side effects. Supplements may also be intentionally spiked with unlisted or illegal 
compounds, or drugs such as anabolic steroids. These are so-called adulterated supplements. 
Supplements containing declared compounds that have not been adequately tested for safety 
can also be declared adulterated by the US Food and Drug Administration (FDA). It has 
been argued that adulterated supplements shouldn’t be considered real dietary supplements 
(64). Yet, such supplements exist and can readily be obtained e.g. over the internet.


page:  8
compounds, DM

In [9]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {'k':3, 'fetch_k':20, 'score_threshold': 0.8, 'lambda_multi':1})
answer = retriever.invoke(question)

In [10]:
answer

[Document(metadata={'source': 'docs\\health supplements\\3.health_supplements_side_effects.pdf', 'page': 6}, page_content='supplements is currently around 20%. Furthermore, bodybuilding and weight loss \nsupplements account for almost half of these injuries (62). Among emergency department \nvisits for adverse events related to dietary supplements in the US, around 25% were due to \nweight loss products (63). There are two classes of adverse effects that may occur. \nSupplements can have components according to the product description that cause certain \nside effects. Supplements may also be intentionally spiked with unlisted or illegal \ncompounds, or drugs such as anabolic steroids. These are so-called adulterated supplements. \nSupplements containing declared compounds that have not been adequately tested for safety \ncan also be declared adulterated by the US Food and Drug Administration (FDA). It has \nbeen argued that adulterated supplements shouldn’t be considered real dietary 

In [11]:
for a in answer:
    print('page: ', a.metadata.get('page'))
    print(a.page_content)
    print('\n')

page:  6
supplements is currently around 20%. Furthermore, bodybuilding and weight loss 
supplements account for almost half of these injuries (62). Among emergency department 
visits for adverse events related to dietary supplements in the US, around 25% were due to 
weight loss products (63). There are two classes of adverse effects that may occur. 
Supplements can have components according to the product description that cause certain 
side effects. Supplements may also be intentionally spiked with unlisted or illegal 
compounds, or drugs such as anabolic steroids. These are so-called adulterated supplements. 
Supplements containing declared compounds that have not been adequately tested for safety 
can also be declared adulterated by the US Food and Drug Administration (FDA). It has 
been argued that adulterated supplements shouldn’t be considered real dietary supplements 
(64). Yet, such supplements exist and can readily be obtained e.g. over the internet.


page:  8
compounds, DM

In [12]:
answer = retriever.invoke('how to lose weight?')
for a in answer:
    print('page: ', a.metadata.get('page'))
    print(a.page_content)
    print('\n')

page:  6
supplements is currently around 20%. Furthermore, bodybuilding and weight loss 
supplements account for almost half of these injuries (62). Among emergency department 
visits for adverse events related to dietary supplements in the US, around 25% were due to 
weight loss products (63). There are two classes of adverse effects that may occur. 
Supplements can have components according to the product description that cause certain 
side effects. Supplements may also be intentionally spiked with unlisted or illegal 
compounds, or drugs such as anabolic steroids. These are so-called adulterated supplements. 
Supplements containing declared compounds that have not been adequately tested for safety 
can also be declared adulterated by the US Food and Drug Administration (FDA). It has 
been argued that adulterated supplements shouldn’t be considered real dietary supplements 
(64). Yet, such supplements exist and can readily be obtained e.g. over the internet.


page:  8
compounds, DM

# RAG with llama on Ollama

In [13]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain import hub

In [14]:
prompt = hub.pull('rlm/rag-prompt')
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [39]:
prompt = """
You are an assistant for question-answering tasks. Use the folloing pieces of retrieve context to answer the question.
If you dont know the answer, just say I don't know.
Use three sentences maximum and keep the answer concise.
Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [40]:
llm = ChatOllama(model='phi3', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content="Hello! How can I help you today? If you're just greeting, that's perfectly fine. Remember to always stay safe and take care of your well-being.", additional_kwargs={}, response_metadata={'model': 'phi3', 'created_at': '2024-11-20T16:59:47.712872Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 663623500, 'load_duration': 12055800, 'prompt_eval_count': 10, 'prompt_eval_duration': 238070000, 'eval_count': 39, 'eval_duration': 412091000}, id='run-55f1ee08-f5dd-4e50-8be6-7c6b941f3133-0', usage_metadata={'input_tokens': 10, 'output_tokens': 39, 'total_tokens': 49})

In [41]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

context = format_docs(answer)

In [42]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
question = 'how to lose weight?'
response = rag_chain.invoke(question)

In [44]:
len(response)

796

In [45]:
print(response)

- Adverse effects from dietary supplements can include liver injury and unexpected drug interactions, such as with anti-obesity drugs like sibutramine or rimonabant. These side effects might occur due to unintended components in adulterated products containing banned substances.
- The FDA has found that some supplements contain illegal compounds intended for weight loss, such as fluoxetine (Prozac), which could lead to harmful psychological and physiological effects when used with dietary supplements marketed for bodybuilding or fat reduction.
- Some brands of multi-ingredient supplements have been linked to serious health issues like liver damage, often attributed to undeclared drug contents such as sibutramine analogues in Hydroxycut products from earlier versions containing ephedra.


In [46]:
question = 'how to gain muscle mass?'
response = rag_chain.invoke(question)
print(response)

- Use steroids under professional supervision to safely increase muscle mass while minimizing health risks. Avoid self-administration of anabolic steroids due to potential for severe side effects and legal issues, as they may result in adverse reactions similar to the use of anti-obesity drugs found in weight loss supplements (96).

- Opt for natural bodybuilding methods such as proper nutrition rich in proteins, balanced diet with adequate calories and a well-designed exercise regimen. This approach can help achieve muscle growth without resorting to potentially harmful substances or practices often found in the supplement industry (96).
